# BERT For Maked Language Model

In [22]:
// Download resources
// config file
//$wget "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json"
// weights
//$wget -O "bert-base-uncased.gt" "bert-base-uncased-model.gt" "https://cdn.huggingface.co/bert-base-uncased-rust_model.ot"

ERROR: repl.go:3:1: undefined identifier: ls

In [36]:
import(
    "fmt"
    
    ts "github.com/sugarme/gotch/tensor"
    "github.com/sugarme/gotch/nn"
    "github.com/sugarme/gotch"
    "github.com/sugarme/tokenizer"
    "github.com/sugarme/tokenizer/pretrained"
    "github.com/sugarme/transformer"
    "github.com/sugarme/transformer/bert"
)

In [12]:
// load pretrained bert-base-uncased tokenizer
tk := pretrained.BertBaseUncased()

In [33]:
device := gotch.CPU
vs := nn.NewVarStore(device)
// load BERT config
config, err := bert.ConfigFromFile("/home/sugarme/projects/transformer/data/bert/bert-base-uncased-config.json")
if err != nil{
    fmt.Print(err)
}
model := bert.NewBertForMaskedLM(vs.Root(), config)
// load weights for BERT masked language model
err := vs.Load("/home/sugarme/projects/transformer/data/bert/model.ot")
if err != nil{
    fmt.Print(err)
}

In [56]:
// Input sample
sentence := "It was a very nice and [MASK] day."
// Encode the input
enc, err := tk.EncodeSingle(sentence, true)
if err != nil{fmt.Print(err)}
var tokInput []int64
for _, id := range enc.Ids{
    tokInput = append(tokInput, int64(id))
}
// Create input tensors from token Ids
// NOTE: BERT model is designed to take multiple samples.
tokTensors := []ts.Tensor{*ts.TensorFrom(tokInput)} 
inputTs, err := ts.Stack(tokTensors, 0)
if err != nil{fmt.Print(err)}
input, err := inputTs.To(device, true)
if err != nil { fmt.Print(err) }

// Forward through the model
var output *ts.Tensor
ts.NoGrad(func(){
    output, _, _ = model.ForwardT(input, ts.None, ts.None, ts.None, ts.None, ts.None, ts.None, false)
})

// Get first sample
output1, err := output.Get(0)
if err != nil{fmt.Print(err)}
// Get p-values for [MASK] token
values, err := output1.Get(7)
if err != nil{ fmt.Print(err) }
// Get the best value
am, err := values.Argmax([]int64{0}, false, false)
if err != nil{ fmt.Print(err) }
id := am.Int64Values()[0]
// Lookup token from vocab
if word, ok := tk.IdToToken(int(id)); !ok { fmt.Printf("Token not found for input Id: %v\n", id)}

fmt.Printf("Input: %v - Output: %v\n", sentence, word)

Input: It was a very nice and [MASK] day. - Output: sunny


58 <nil>